# Purpose - look at fsspec module

## and inrto to geoviews gv

https://filesystem-spec.readthedocs.io/en/latest/api.html

In [ ]:
import fsspec
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)

path_cog='eco-w1/in1/rapv2_cog'
path_lzw='eco-w1/in0/rapv2'

if(fs.isdir(path_cog)):
    print('YAY!')
if(fs.isdir(path_lzw)):
    print('YAY!')

In [ ]:
cog_disk_usage = fs.disk_usage(path_cog)

# Cogs use more space
- but its worth it!

In [ ]:
cog_disk_usage/1e9

In [ ]:
lzw_disk_usage = fs.disk_usage(path_lzw)
lzw_disk_usage/1e9

In [ ]:
# Cogs have overviews and use deflate

cog_tifs = fs.ls(path_cog)
cog_tifs[0]

In [ ]:
lzw_tifs = fs.ls(path_lzw)
lzw_tifs[0]

In [ ]:
import rasterio
src = rasterio.open(f's3://{cog_tifs[0]}', 'r')
[src.overviews(i) for i in src.indexes]

In [ ]:
src.shape

In [ ]:
def ov_sz(src):
    print('Full Height: ',src.height)
    print('Full, Width:',src.width)
    for i in src.indexes:
        scales = src.overviews(i)
        print(scales)
        for scale in scales:
            print((int(src.height/scale),int(src.width/scale)))
    

In [ ]:
ov_sz(src)

https://lpdaac.usgs.gov/resources/e-learning/getting-started-cloud-native-hls-data-python/

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

https://eric.clst.org/tech/usgeojson/

- 500K scale states

In [ ]:
#! wget https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_500k.json

In [ ]:
#! head gz_2010_us_040_00_500k.json

In [ ]:
import json


In [ ]:
# data=json.load('gz_2010_us_040_00_500k.json')

gdf = gpd.read_file('gz_2010_us_040_00_500k.json')
# gdf = gpd.GeoDataFrame(data)
print(gdf.head())

# features column does not need to be stored, this is just for illustration
gdf['features'] = gdf['geometry'].apply(lambda x: {'geometry': x, 'properties': {}})
gdf2 = gpd.GeoDataFrame.from_features(gdf['features'])

gdf = gdf.set_geometry(gdf2.geometry)
gdf.head()

In [ ]:
state=gdf['geometry'][4]

In [ ]:
state

In [ ]:
# stateShape = state['geometry'][0] # Define the geometry as a shapely polygon
# stateShape

In [ ]:
import geoviews as gv
gv.extension('bokeh', 'matplotlib')
# Use geoviews to combine a basemap with the shapely polygon of our Region of Interest (ROI)
base = gv.tile_sources.EsriImagery.opts(width=650, height=500)
nevada = gv.Polygons(state).opts(line_color='yellow', color=None)
base * nevada

In [ ]:
#! pip install --user geoviews

In [ ]:
#! pip install --user --upgrade --force-reinstall shapely cartopy --no-binary shapely --no-binary cartopy 

In [ ]:
cog_tifs[0]

In [ ]:
import rasterio
import xarray as xr

#returns an xarray DA
def overview_ugly_xr_read(s3_tif_object):
    da = xr.open_rasterio(f's3://{s3_tif_object}')
    src = rasterio.open(f's3://{s3_tif_object}', 'r')
    #nsub = 4  # original scale factor
    #nsub = 64
    nsub=128
    nsub_half = int(nsub/2)
    data = src.read(out_shape=(1, int(src.height / nsub), int(src.width / nsub)))
    DA = xr.DataArray(data, dims=da.dims, coords=[da.band, da.y[nsub_half:-nsub_half:nsub], da.x[nsub_half:-nsub_half:nsub]])
    DA.attrs = da.attrs
    return(DA)

In [ ]:
da = overview_ugly_xr_read(cog_tifs[0])

In [ ]:
da

In [ ]:
da_masked = da.where(da<50.0)
da_masked.plot(cmap='OrRd')

In [ ]:
type(da_masked.values)

In [ ]:
from rasterio.mask import mask
nev_array, nev_transform = rasterio.mask.mask(da, [state], crop=True, nodata=None)